In [1]:
import sys
sys.path.append("/Users/david/Documents/mastersCode/ubm")
print(sys.path)

['/Users/david/Documents/mastersCode/ubm/classifiers/hmm_nn/dnnhmm/development', '/Users/david/.conda/envs/ubm/lib/python310.zip', '/Users/david/.conda/envs/ubm/lib/python3.10', '/Users/david/.conda/envs/ubm/lib/python3.10/lib-dynload', '', '/Users/david/.conda/envs/ubm/lib/python3.10/site-packages', '/Users/david/Documents/mastersCode/ubm']


In [2]:
from importlib import reload
import audio_datastore.audio_datastore as myads
reload(myads)
from importlib import reload
from collections import Counter
import pickle
import time

Using cache found in /Users/david/.cache/torch/hub/snakers4_silero-vad_master
Using cache found in /Users/david/.cache/torch/hub/snakers4_silero-vad_master


In [3]:
from data.pitch_tracking_database.load_data import *
speech_pitch_tracking_db = pickle.load(open(normalised_30,'rb'))
ads = speech_pitch_tracking_db['ads']

In [4]:
# split ads
developmentLabels = ["M03","M04","M05","M06","M07","M08","M09","M10","F05","F06","F07","F08","F09","F10",
                     "F03","F04"]
evaluationLabels = ["M01","M02","F01","F02"]
adsTrain = myads.subset(ads, developmentLabels)
adsEvaluate = myads.subset(ads, evaluationLabels)
numFilesPerSpeakerForEnrollment = 4
adsEnroll, adsDet = myads.split(adsEvaluate, numFilesPerSpeakerForEnrollment)
adsTest = adsDet
# adsDet, adsTest = myads.split(adsDet, 2)
# adsTrain.info('Train')
# adsEnroll.info('Enroll')
# adsTest.info('Test')

In [7]:
from classifiers.classifier_base import ClassifierBase
from my_torch.tuts2.torch_transforms import ComposeTransform

def test_classifiers(ads_train, ads_test, classifiers: [ClassifierBase], verbose=False):
    index = 1
    if verbose:
        print('running all')

    for classifier in classifiers:
        if verbose:
            print('\n')
            print('test #', index)
        print(classifier.info)

        classifier.train(ads_train)
        classifier.test_all(ads_test)
        index = index + 1
        if verbose:
            print('\n')

In [8]:
import classifiers.hmm_nn.dnnhmm.classifier_dnn_hmm as classifier_dnn_hmm
reload(classifier_dnn_hmm)

import my_torch.tuts2.torch_transforms as torch_t
reload(torch_t)
from feature_extraction.fe_spafe.fe_spafe import FeatureExtractorMfcc

# define fe methods

fe_mfcc = FeatureExtractorMfcc()
fe_mfcc.set_normalisation(adsDet)


In [9]:
# define classifiers

classifier_a = classifier_dnn_hmm.ClassifierHMMDNN(
    train_process=torch_t.ComposeTransform([
        torch_t.FeMethodCheck(),
        fe_mfcc
    ]),
    test_process=torch_t.ComposeTransform([
    torch_t.FeMethodCheck(),
    fe_mfcc,
    ]),
    fe_method=torch_t.ComposeTransform([
    fe_mfcc,
    ]),
    info='base mfcc no augmentation'
)

In [10]:
test_classifiers(adsEnroll, adsTest, [classifier_a], verbose=True)

running all


test # 1
base mfcc no augmentation
input vector shape incorrect
input vector shape incorrect
input vector shape incorrect
input vector shape incorrect


ValueError: Input contains NaN.